In [1]:
!pip install -U -q tqdm json_repair ddgs requests beautifulsoup4 SQLAlchemy wikipedia-api

In [2]:
import requests
import zipfile
import io
import sys
import os
import shutil

# 1. The URL for the source code ZIP
url = "https://github.com/anpc849/kagentic/archive/refs/heads/main.zip"

# 2. Download the file using requests
print("Downloading...")
response = requests.get(url)
if response.status_code == 200:
    # 3. Unzip the content directly from memory
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        z.extractall(".")
    print("Unzipped successfully!")
else:
    print(f"Failed to download. Status code: {response.status_code}")

# 4. Add the folder to the Python Path
# GitHub's ZIPs always create a folder named 'repo-branch'
shutil.move(os.path.abspath('kagentic-main'), os.path.abspath('kagentic'))
module_path = "/kaggle/working"
if module_path not in sys.path:
    sys.path.append(module_path)

# 5. Verify the import
try:
    import kagentic
    print("Import successful! You can now use kagents.")
except ImportError:
    print("Import failed. Try checking !ls kagentic to see the folder structure.")

Downloading...
Unzipped successfully!


Import successful! You can now use kagents.


In [5]:
import kaggle_benchmarks as kbench
from kagentic import CodeAgent, Tool, ToolInput

In [6]:
from sqlalchemy import (
    Column,
    Float,
    Integer,
    MetaData,
    String,
    Table,
    create_engine,
    insert,
    inspect,
    text,
)

# ---------------------------------------------------------------------------
# Setup in-memory SQLite DB
# ---------------------------------------------------------------------------
engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

receipts = Table(
    "receipts",
    metadata_obj,
    Column("receipt_id", Integer, primary_key=True),
    Column("customer_name", String(16), primary_key=True),
    Column("price", Float),
    Column("tip", Float),
)
metadata_obj.create_all(engine)

rows = [
    {"receipt_id": 1, "customer_name": "Alan Payne",      "price": 12.06, "tip": 1.20},
    {"receipt_id": 2, "customer_name": "Alex Mason",       "price": 23.86, "tip": 0.24},
    {"receipt_id": 3, "customer_name": "Woodrow Wilson",   "price": 53.43, "tip": 5.43},
    {"receipt_id": 4, "customer_name": "Margaret James",   "price": 21.11, "tip": 1.00},
]
for row in rows:
    with engine.begin() as conn:
        conn.execute(insert(receipts).values(**row))

# Build table description for the prompt
inspector = inspect(engine)
columns_info = [(col["name"], col["type"]) for col in inspector.get_columns("receipts")]
table_description = "Columns:\n" + "\n".join(
    f"  - {name}: {col_type}" for name, col_type in columns_info
)
print(table_description)

Columns:
  - receipt_id: INTEGER
  - customer_name: VARCHAR(16)
  - price: FLOAT
  - tip: FLOAT


In [7]:
# ---------------------------------------------------------------------------
# SQL tool (kagents-style)
# ---------------------------------------------------------------------------
class SQLEngineTool(Tool):
    name = "sql_engine"
    description = (
        "Executes a SQL query against the 'receipts' table and returns the result. "
        f"Table schema:\n{table_description}"
    )
    inputs = {
        "query": ToolInput(
            type="string",
            description="A valid SQL SELECT query to run against the 'receipts' table.",
            required=True,
        )
    }
    output_type = "string"

    def forward(self, query: str) -> str:
        output = ""
        with engine.connect() as con:
            rows = con.execute(text(query))
            for row in rows:
                output += "\n" + str(row)
        return output.strip() or "(no rows returned)"

In [10]:
@kbench.task(
    name="sql_tool_multihop_reasoning",
    description="Evaluate whether the agent performs multi step reasoning and uses the SQL tool to retrieve and combine database information."
)
def text_to_sql_task(llm):

    question = (
        "Using SQL, compute the total payment for each customer "
        "(price plus tip). Find the difference between the highest "
        "total payment and the second highest total payment. "
        "Return only the numeric difference rounded to two decimal places."
    )
    
    agent = CodeAgent(
        tools=[SQLEngineTool()],
        model=llm,
        max_steps=5,
        verbosity_level=2,
        additional_instructions=(
            "You have access to an in-memory SQLite database. "
            "Use sql_engine to query it. Write correct SQL only."
        ),
    )

    print(f"\nQuestion: {question}\n")

    answer = agent.run(question)

    print("\n" + "=" * 60)
    print("FINAL ANSWER:")
    print(answer)
    print("=" * 60)

    # Expected numeric result
    expected_difference = "34.76"

    kbench.assertions.assert_true(
        expected_difference in answer,
        expectation="Answer should contain the correct payment difference of 34.76."
    )


In [11]:
text_to_sql_task.run(kbench.llm)


Question: Using SQL, compute the total payment for each customer (price plus tip). Find the difference between the highest total payment and the second highest total payment. Return only the numeric difference rounded to two decimal places.

[agent] 
[agent] 🤖 kagentic starting — model: google/gemini-2.5-flash
[agent] 📋 Task: Using SQL, compute the total payment for each customer (price plus tip). Find the difference between the highest total p...
[agent] ============================================================


[agent] --- Step 1 ---
[agent]   🎯 Action:  sql_engine({"query": "WITH CustomerTotals AS (\n    SELECT\n        customer_name,\n        SUM(price + tip) AS total_payment\n    FROM\n        receipts\n    GROUP BY\n        customer_name\n)\nSELECT\n    ROUND(\n        (SELECT total_payment FROM CustomerTotals ORDER BY total_payment DESC LIMIT 1 OFFSET 0) -\n        (SELECT total_payment FROM CustomerTotals ORDER BY total_payment DESC LIMIT 1 OFFSET 1),\n        2\n    );"})

BokehModel(combine_events=True, render_bundle={'docs_json': {'e4a28a68-6495-4352-b0c8-5fa192fdaf79': {'version…